In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime
import numpy as np
import cProfile
import sys
import pickle
import random

from graph import TransportGraph
from forward_search import FCH

# Build transport graph

In [2]:
CITY = 'kuopio' #belfast, kuopio

## Optional 
### Add inverted road connections

In [4]:
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')

In [4]:
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

In [5]:
tg = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

In [6]:
len(tg.nodes)

549

# Build CH graph

In [7]:
%%time
cProfile.run('ch_tg = tg.contraction_hierarchy(just_buses=True)')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:04<00:00, 128.97it/s]

         21405096 function calls (18668362 primitive calls) in 12.109 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:100(acquire)
     10/3    0.000    0.000    0.019    0.006 <frozen importlib._bootstrap>:1022(_find_and_load)
     14/7    0.000    0.000    0.009    0.001 <frozen importlib._bootstrap>:1053(_handle_fromlist)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:125(release)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:165(__init__)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:169(__enter__)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:173(__exit__)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:179(_get_module_lock)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:

### Calculate Full-CH

In [9]:
%%time
cProfile.run('ch_tg_optimal = tg.contraction_hierarchy(just_buses=False)')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [05:21<00:00,  1.71it/s]

         393666588 function calls (390929913 primitive calls) in 330.574 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  330.573  330.573 <string>:1(<module>)
    18896    0.034    0.000    0.441    0.000 _collections_abc.py:957(pop)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:991(update)
        1    0.000    0.000    0.000    0.000 _monitor.py:94(report)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:111(remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:17(__init__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:21(__enter__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:27(__exit__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:53(_commit_removals)
        3    0.000    0.000    0.000    0.000 _weakrefset.py:63(__iter__)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:86(add)
      

%%time
cProfile.run('ch_tg = tg.contraction_hierarchy')

# Precalculate Geometrical containers for FCH

In [10]:
%%time
ch_tg_optimal.geometrical_container()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:00<00:00, 1699.95it/s]

CPU times: user 215 ms, sys: 42.5 ms, total: 257 ms
Wall time: 333 ms


In [11]:
pickle.dump(ch_tg_optimal, open(F'kuopio_original.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

In [12]:
pickle.dump(ch_tg, open(F'kuopio_trunc.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

In [7]:
ch_tg = pickle.load(open(F'kuopio_trunc.pkl', 'rb'))
ch_tg_optimal = pickle.load(open(F'kuopio_original.pkl', 'rb'))

# Pathfinding

In [8]:
transport_connections['dep_time_ut'].min(), transport_connections['dep_time_ut'].max()

(1481511600, 1481585460)

In [10]:
test_pairs = pd.DataFrame([
    {'start_time': 1481514083, 'start_node': 330, 'end_node': 54},
    {'start_time': 1481519880, 'start_node': 345, 'end_node': 141},
    {'start_time': 1481518900, 'start_node': 1, 'end_node': 132},
    {'start_time': 1481514540, 'start_node': 116, 'end_node': 118},
    {'start_time': 1481519640, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 51},
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481571574, 'start_node': 146, 'end_node': 7}
])
for index, row in test_pairs.iterrows():
    pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60)
    print('Path duration', datetime.timedelta(seconds=int(path['arrival'] - row['start_time'])))
    print('Calculation duration', datetime.timedelta(seconds=int(path['duration'])))
    print()

Path duration 0:55:37
Calculation duration 0:00:34

Path duration 0:05:00
Calculation duration 0:00:06

Path duration 0:27:20
Calculation duration 0:01:10

Path duration 0:00:00
Calculation duration 0:00:00

Path duration 0:07:00
Calculation duration 0:00:08

Path duration 0:05:55
Calculation duration 0:00:04

Path duration 0:04:20
Calculation duration 0:00:03

Path duration 0:22:26
Calculation duration 0:00:34



In [9]:
test_pairs = pd.DataFrame([
    {'start_time': 1481514083, 'start_node': 330, 'end_node': 54},
    {'start_time': 1481519880, 'start_node': 345, 'end_node': 141},
    {'start_time': 1481518900, 'start_node': 1, 'end_node': 132},
    {'start_time': 1481514540, 'start_node': 116, 'end_node': 118},
    {'start_time': 1481519640, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 51},
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481571574, 'start_node': 146, 'end_node': 7}
])
for index, row in test_pairs.iterrows():
    pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, geometrical_containers=False)
    print('Path duration', datetime.timedelta(seconds=int(path['arrival'] - row['start_time'])))
    print('Calculation duration', datetime.timedelta(seconds=int(path['duration'])))
    print()

Path duration 0:55:37
Calculation duration 0:00:42

Path duration 0:05:00
Calculation duration 0:00:05

Path duration 0:27:20
Calculation duration 0:00:44

Path duration 0:00:00
Calculation duration 0:00:00

Path duration 0:07:00
Calculation duration 0:00:08

Path duration 0:05:55
Calculation duration 0:00:04

Path duration 0:04:20
Calculation duration 0:00:03

Path duration 0:22:26
Calculation duration 0:00:20



# Compare 2 solutions

In [41]:
N = 1_000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]
             })
new_duration = []
optimal_duration = []
for index, row in test_data.iterrows():
    
    # new search
    pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False)
    if path['path']:
        new_duration.append(path['duration'])
    new_arrival = path['arrival']
    
    # optimal_duration
    pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True)
    if path['path']:
        optimal_duration.append(path['duration'])
    optimal_arrival = path['arrival']
    
    # validate
    assert new_arrival == optimal_arrival

/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_96783/912335882.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)],
/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_96783/912335882.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]


In [42]:
np.mean(optimal_duration), np.median(optimal_duration), np.std(optimal_duration)

(12.350276243093923, 12.0, 6.819533403048331)

In [43]:
np.mean(new_duration), np.median(new_duration), np.std(new_duration)

(30.609944751381214, 33.0, 15.269776977651762)

### Withoout geometrical containers

In [44]:
N = 1_000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]
             })
new_duration = []
optimal_duration = []
for index, row in test_data.iterrows():
    
    # new search
    pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False)
    if path['path']:
        new_duration.append(path['duration'])
    new_arrival = path['arrival']
    
    # optimal_duration
    pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=False)
    if path['path']:
        optimal_duration.append(path['duration'])
    optimal_arrival = path['arrival']
    
    # validate
    assert new_arrival == optimal_arrival

/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_96783/2655839994.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)],
/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_96783/2655839994.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]


In [45]:
np.mean(optimal_duration), np.median(optimal_duration), np.std(optimal_duration)

(35.20987654320987, 37.0, 17.954917764400847)

In [46]:
np.mean(new_duration), np.median(new_duration), np.std(new_duration)

(33.57575757575758, 35.0, 17.665225669064796)

# Visualize path

In [8]:
%%time
pathfinding = FCH(graph=ch_tg,
                      start_time=1481519880,
                      start_node=345, 
                      end_node=141)
path = pathfinding.shortest_path(60)

CPU times: user 4.06 ms, sys: 1.18 ms, total: 5.24 ms
Wall time: 6.07 ms


In [9]:
nodes = pd.read_csv(F'data/{CITY}/network_nodes.csv', sep=';')
nodes_dict = nodes.set_index('stop_I').to_dict('index')

In [12]:
for station in path['path']:
    print(station, nodes_dict[station]['lat'], nodes_dict[station]['lon'], nodes_dict[station]['name'])

345 62.894633 27.645134 Canthia E
114 62.891878 27.637245 Snellmania E
116 62.88952 27.632591 Teknia E
118 62.888191 27.630635 Technopolis E
141 62.88333 27.61765 Neulamäentie 5 L


In [13]:
i = 0
k = 0
while i < len(path['routes']):
    x = path['routes'][i]
    if k == 0:
        i += 1
        if i == len(path['routes']):
            fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines",
            lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
            lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
            text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
            name = path['routes'][k],
            marker = {'size': 10}))
            k = i
        elif x != path['routes'][i]:
            fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines",
            lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
            lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
            text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
            name = path['routes'][k],
            marker = {'size': 10}))
            k = i
    else:
        i += 1
        if i < len(path['routes']):
            if x != path['routes'][i]:
                fig.add_trace(go.Scattermapbox(
                mode = "markers+lines",
                lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
                lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
                text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
                name = path['routes'][k],
                marker = {'size': 10}))
                k = i
        else:
            fig.add_trace(go.Scattermapbox(
            mode = "markers+lines",
            lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
            lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
            text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
            name = path['routes'][k],
            marker = {'size': 10}))
            k = i
        

fig.update_layout(
    mapbox = {
        'style': "stamen-terrain",
        'center': {'lon': nodes['lon'].iloc[0], 'lat': nodes['lat'].iloc[0]},
        'zoom': 9}, 
    showlegend=True)

fig.show()